In [104]:
!pip install pylatexenc
!pip install qiskit-aer
import qiskit
import qiskit_ionq

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [146]:
import numpy as np
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider (
    resource_id = "/subscriptions/9f94f9e5-0590-4d7d-8d62-d59a9fd5073e/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/Elias-Lehman-Quantum",
    location = "westus"
)
from qiskit import assemble, Aer
from qiskit import QuantumCircuit
import matplotlib.pyplot as plt
from qiskit.tools.monitor import job_monitor
import qiskit.quantum_info as qi

print("This workspace's targets:")
for backend in provider.backends():
    print("- " + backend.name())

This workspace's targets:
- ionq.qpu
- ionq.simulator
- quantinuum.hqs-lt-s1
- quantinuum.hqs-lt-s1-apival
- quantinuum.hqs-lt-s2
- quantinuum.hqs-lt-s2-apival
- quantinuum.hqs-lt-s1-sim
- quantinuum.hqs-lt-s2-sim


In [147]:
#Gets X-gate error of particular backend
def getXerr(target, shots):
    xcircuit = QuantumCircuit(1,1)

    #We expect X on |0> to output |1>
    xcircuit.x(0)
    xcircuit.measure(0,0)

    backend = provider.get_backend(target)
    job = backend.run(xcircuit, count=shots)
    print("Job id:", job.id())
    results = job.result()
    counts = results.get_counts()
    return (1-counts['1']/shots)

#Getting Y-gate error 
def getYerr(target, shots):
    ycircuit = QuantumCircuit(1,1)

    #We look for bit-flip errors in Y
    ycircuit.y(0)
    ycircuit.measure(0,0)

    backend = provider.get_backend(target)
    job = backend.run(ycircuit, count=shots)
    print("Job id:", job.id())
    results = job.result()
    counts = results.get_counts()
    return (1-counts['0']/shots)

#Getting Z-gate error
def getZerr(target, shots):
    zcircuit = QuantumCircuit(1,1)

    #We expect HZH on |0> to output -|1>
    zcircuit.h(0)
    zcircuit.z(0)
    zcircuit.h(0)
    zcircuit.measure(0,0)

    backend = provider.get_backend(target)
    job = backend.run(zcircuit, count=shots)
    print("Job id:", job.id())
    results = job.result()
    counts = results.get_counts()
    return (1-counts['1']/shots)

In [148]:
#Get density matrix
def getRho(qc):
    rho = qi.DensityMatrix.from_instruction(qc)
    return rho

In [149]:
def NoiseFinder( qc, backend, gateSampleSize):
    #Let's define the 2^n-1 X 2^n-1 Pauli matrices
    X = np.matrix([[0,1],[1,0]])
    Y = np.matrix([[0,-1j],[1j,0]])
    Z = np.matrix([[1,0],[0,-1]])
    for n in range(qc.num_qubits()):
        X = np.tensordot(X, X, 0)
        Y = np.tensordot(Y, Y, 0)
        Z = np.tensordot(Z, Z, 0)


    print("Backend error estimates of Pauli gates:")
    e_x = getXerr(backend, gateSampleSize)
    print(f"e_x: {e_x}")
    e_y = getYerr(backend, gateSampleSize)
    print(f"e_y: {e_y}")
    e_z = getZerr(backend, gateSampleSize)
    print(f"e_z: {e_z}")

    from qiskit.visualization import plot_state_city

    #Get State Density Matrix:
    rho = getRho(qc)

    #Get Generalized Noise:
    Xi = rho._multiply(1 - e_x - e_y - e_z) + e_x*np.matmul(X, np.matmul(rho, X.H())) + e_y*Y.tensor(rho.tensor(Y.H())) + e_z*Z.tensor(rho.tensor(Z.H()))
    plot_state_city(Xi.data, title='Generalized Noise')

In [103]:
#Define custom Quantum Circuit using Pauli gates below.
# For Example:
qc = QuantumCircuit(1)
qc.x(0)
qc.z(0)
## Add Gates Here ##

NoiseFinder(qc, "ionq_qpu", 1000)

Backend error estimates of Pauli gates:
Job id: fec35b6c-b8ba-11ec-a603-00155d0c6290
..................................................................................

In [102]:
#This cell is for automated testing on multiple QPUs. 
#Uncomment the lines below to test both of Azur's availble quantum computers.
#WARNING: May take a while...

#Iterates through backends and prints results.
"""
backends=["quantinuum.hqs-lt-s2", "ionq_qpu"]
for qpu in backends:
    getXerr(qpu)
"""

'\nbackends=["quantinuum.hqs-lt-s2", "ionq_simulator"]\nfor qpu in backends:\n    getXerr(qpu)\n'